This code runs version 1.4.1 of the FaIR climate model and is built off the temperature-attribution repository developed by Chris Smith: https://github.com/chrisroadmap/temperature-attribution/tree/main

To set up an environment to run the code:

1) clone the temperature-attribution repository and cd to temperature-attribution
2) create the environment using conda env create -f environment.yml (optional if needed)
3) activate the environment with conda activate temperature-attribution (optional if needed)
4) run nbstripout --install to ensure clean notebook commits (optional if needed)
5) download this jupyter notebook and put the notebook in the notebooks directory

In [1]:
#Import required packages

import numpy as np
import pandas as pd
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [2]:
import importlib.resources as pkg_resources

In [3]:
#Define functions for the different FaIR runs and analyses

def run_fair(emissions_file, forcing_file, scenarios, adjust = False, emissions_adj_co2 = np.array([]), emissions_adj_ch4 = np.array([])):
    #f = FAIR()
    f = FAIR(ch4_method="Thornhill2021")
    f.define_time(1750, 2500, 1)  # start, end, step
    f.define_scenarios(scenarios)
    fair_params_1_4_1_file = "../data/original/calibrated_constrained_parameters_calibration1.4.1.csv"  #'../data/calibrated_constrained_ensemble/calibrated_constrained_parameters_calibration1.4.1.csv'
    df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
    configs = df_configs.index  # this is used as a label for the "config" axis
    f.define_configs(configs)
    fair_species_configs_1_4_1_file =  "../data/original/species_configs_properties_calibration1.4.1.csv"#'../data/calibrated_constrained_ensemble/species_configs_properties_calibration1.4.1.csv'
    species, properties = read_properties(filename=fair_species_configs_1_4_1_file)
    f.define_species(species, properties)
    f.allocate()
    f.fill_from_csv(
        emissions_file='../data/original/'+emissions_file,
        forcing_file='../data/original/'+forcing_file,
    )

    #Code to add a custom array to the default emissions data for both CO2 and CH4
    if adjust == True:
        time_mask   = f.emissions['timepoints'] >= 2025
        time_sel    = f.emissions['timepoints'].where(time_mask, drop=True)
        n_years     = time_sel.size
        
        adj_trimmed_co2 = emissions_adj_co2[:n_years]
        adj_da_co2 = xr.DataArray(
            adj_trimmed_co2,
            coords={'timepoints': time_sel},
            dims='timepoints'
        )
        co2_slice = dict(timepoints=time_sel, specie='CO2 FFI')
        
        f.emissions.loc[co2_slice] = (
            f.emissions.sel(**co2_slice) + adj_da_co2
        )

        adj_trimmed_ch4 = emissions_adj_ch4[:n_years]
        adj_da_ch4 = xr.DataArray(
            adj_trimmed_ch4,
            coords={'timepoints': time_sel},
            dims='timepoints'
        )
        ch4_slice = dict(timepoints=time_sel, specie='CH4')

        f.emissions.loc[ch4_slice] = (
            f.emissions.sel(**ch4_slice) + adj_da_ch4
        )
    
    fill(
        f.forcing,
        f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
        specie="Volcanic",
    )
    fill(
        f.forcing,
        f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
        specie="Solar",
    )
    f.fill_species_configs(fair_species_configs_1_4_1_file)
    f.override_defaults(fair_params_1_4_1_file)
    initialise(f.concentration, f.species_configs["baseline_concentration"])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)
    initialise(f.airborne_emissions, 0)
    initialise(f.ocean_heat_content_change, 0)
    f.run()
    return f


def diffs_export_csv(f, ff, scenario_selected, label_name, ylabel_name, csv_file):
    # ensure scalar scenario (works if user passed ['low'])
    if not isinstance(scenario_selected, str):
        scenario_selected = np.asarray(scenario_selected).ravel()[0]

    t_ff = ff.temperature.sel(scenario=scenario_selected, layer=0).squeeze(drop=True)
    t_f  =  f.temperature.sel(scenario=scenario_selected, layer=0).squeeze(drop=True)
    temp_diff = t_ff - t_f

    f_ff = ff.forcing_sum.sel(scenario=scenario_selected).squeeze(drop=True)
    f_f  =  f.forcing_sum.sel(scenario=scenario_selected).squeeze(drop=True)
    forcing_diff = f_ff - f_f

    def qdf(diff):
        # Compute all three at once; remaining dim is the time-like dim (name-agnostic)
        q = diff.quantile([0.05, 0.50, 0.95], dim="config")
        time_dim = [d for d in q.dims if d != "quantile"][0]
        # Put time on rows, quantiles on columns and convert to pandas
        df = q.transpose(time_dim, "quantile").to_pandas()
        df.columns = ["diffs_05", "diffs_50", "diffs_95"]
        return df

    df_out_temperature = qdf(temp_diff)
    df_out_forcing     = qdf(forcing_diff)

    df_out_temperature.to_csv("../data/outputs/"+csv_file)
    df_out_forcing.to_csv("../data/outputs/forcing_" + csv_file)
    

def difference_scenarios(f, co2_perturbation, ch4_perturbation, scenarios, savenames):
    ff = run_fair('extensions_1750-2500.csv', 'volcanic_solar.csv', scenarios, True, co2_perturbation, ch4_perturbation)
    diffs_export_csv(f, ff, scenarios, 'CDR Temperature Effects', 
                    '°C relative to baseline scenario', savenames+'.csv')

    # handle list-like scenarios robustly
    for scen in np.atleast_1d(scenarios):
        scen = np.asarray([scen]).ravel()[0]   # ensure scalar label
        diffs_export_csv(f, ff, scen,
                         'CDR Temperature Effects',
                         '°C relative to baseline scenario',
                         f'{savenames}_{scen}.csv')

In [4]:
#Baseline model run and constants

#Scenario selected
scenarios = ['low']
n_years = 20  # number of annual pulses in the cumulative experiment

#No CH4 perturbation in these experiments
emissions_only_ch4 = [0] #35.71
ch4_perturbation = np.r_[emissions_only_ch4, np.full(1000, 0)]

#Run baseline model
f = run_fair('extensions_1750-2500.csv', 'volcanic_solar.csv', scenarios)

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

In [5]:
#Scenario 0: Baseline emissions (1 GtCO2)

emissions_co2 = [1]
co2_perturbation_pulse = np.r_[emissions_co2, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(emissions_co2, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'baseline_emissions_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'baseline_emissions_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)


Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

In [6]:
#Scenario 1: Physical delay

#First run for CDR only
cdr_only = [-0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, 
            -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05]
co2_perturbation_pulse = np.r_[cdr_only, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_only, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'physical_delay_cdr_only_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'physical_delay_cdr_only_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)


#Second run for CDR and emissions
cdr_emissions = [0.95, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, 
            -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05]
co2_perturbation_pulse = np.r_[cdr_emissions, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_emissions, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'physical_delay_cdr_emissions_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'physical_delay_cdr_emissions_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

In [7]:
#Scenario 2: Counterfactual delay

#First run for CDR only
cdr_only = [-0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, 
            -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05]
co2_perturbation_pulse = np.r_[cdr_only, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_only, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'counterfactual_delay_cdr_only_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'counterfactual_delay_cdr_only_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)


#Second run for CDR and emissions
cdr_emissions = [0.95, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, 
                -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05]
co2_perturbation_pulse = np.r_[cdr_emissions, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_emissions, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'counterfactual_delay_cdr_emissions_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'counterfactual_delay_cdr_emissions_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

In [8]:
#Scenario 3: Accelerated emissions

#First run for CDR only
cdr_only = [0.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
            -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]
co2_perturbation_pulse = np.r_[cdr_only, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_only, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'accelerated_emissions_cdr_only_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'accelerated_emissions_cdr_only_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)


#Second run for CDR and emissions
cdr_emissions = [1.9, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1,
                -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]
co2_perturbation_pulse = np.r_[cdr_emissions, np.full(1000, 0)]   # long tail of 0s
co2_cumulative = np.convolve(cdr_emissions, np.ones(n_years, dtype=float), mode='full')
co2_perturbation_cumulative = np.r_[co2_cumulative, np.full(1000, 0)]   # long tail of 0s

#Do the pulse run
savenames = 'accelerated_emissions_cdr_emissions_pulse'
difference_scenarios(f, co2_perturbation_pulse, ch4_perturbation, scenarios, savenames)

#Do the cumulative run
savenames = 'accelerated_emissions_cdr_emissions_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

In [9]:
#Scenario 4: Upfront embodied emissions (note, no pulse runs are possible here)

#First run for CDR only
cdr_only = [3, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15 -1.15, -1.15, 
            -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15, -1.15]
co2_perturbation_cumulative = np.r_[cdr_only, np.full(1000, 0)]   # long tail of 0s

#Do the cumulative run
savenames = 'upfront_emissions_cdr_only_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)

cdr_only = [3, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15 -0.15, -0.15, 
            -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15, -0.15]
co2_perturbation_cumulative = np.r_[cdr_only, np.full(1000, 0)]   # long tail of 0s

#Do the cumulative run
savenames = 'upfront_emissions_cdr_emissions_cumulative'
difference_scenarios(f, co2_perturbation_cumulative, ch4_perturbation, scenarios, savenames)

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]

Running 841 projections in parallel:   0%|          | 0/750 [00:00<?, ?timesteps/s]